In [1]:
# main/extract_text_method5_trainset
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.eval_rouge import cal_rouge

from nltk.tokenize import word_tokenize

In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import networkx as nx
from heapq import nlargest

[nltk_data] Downloading package punkt to /Users/kneskung/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import pickle, time
from tqdm import tqdm
from datetime import datetime, timedelta
ts = time.time()
import sentence_transformers as sbert

In [4]:
sbmodel = sbert.SentenceTransformer(model_name_or_path='all-MiniLM-L6-v2', device='cuda')
ts = time.time()

In [5]:
import numpy as np
np.__version__

'1.24.1'

In [6]:
import spacy #2.3.8 with numpy==1.21
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [7]:
import matplotlib.pyplot as plt

In [9]:
import allennlp
allennlp.__version__

'2.10.1'

In [10]:
from generate_summary.intersection_strategy import IntersectionStrategy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/634M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
coref = IntersectionStrategy()

In [12]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [13]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at
back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by
call can cannot ca could
did do does doing done down due during
each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except
few fifteen fifty first five for former formerly forty four from front full
further
get give go
had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred
i if in indeed into is it its itself
keep
last latter latterly least less
just
made make many may me meanwhile might mine more moreover most mostly move much
must my myself
name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere
of off often on once one only onto or other others otherwise our ours ourselves
out over own
part per perhaps please put
quite
rather re really regarding
same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such
take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two
under until up unless upon us used using
various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would
yet you your yours yourself yourselves
cnn ll ve lrb rrb -PRON-
""".split()
)

contractions = ["n't", "'d", "'ll", "'m", "'re", "'s", "'ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))
"""
contractions = ["n't", "'d", "'ll", "'m", "'re", "'s", "'ve"]
STOP_WORDS.update(contractions)"""

'\ncontractions = ["n\'t", "\'d", "\'ll", "\'m", "\'re", "\'s", "\'ve"]\nSTOP_WORDS.update(contractions)'

In [14]:
remove_list = [".", ",", "'", "\"", '``', "'", ',', '-', '`', "''"]

In [15]:
import pandas as pd
news_list = pd.read_pickle("data/sample_news.pkl")

In [16]:
def find_tfidf(content):
    vectorizer = TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(1, 1))
    x = vectorizer.fit_transform(content)
    vectorizer.get_feature_names_out()
    df = pd.DataFrame(x.toarray(),  columns=vectorizer.get_feature_names_out())
    df = df.stack().reset_index()
    df = df.rename(columns={0:'tfidf', 'level_0': 'document', 'level_1': 'term'})
    df2 = df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)
    tf_term = df2['term'].tolist()
    return tf_term

In [17]:
def find_tfidf_idx(idx):
    content = [news_list[idx]['content']]
    vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 1))
    x = vectorizer.fit_transform(content)
    vectorizer.get_feature_names_out()
    df = pd.DataFrame(x.toarray(),  columns=vectorizer.get_feature_names_out())
    df = df.stack().reset_index()
    df = df.rename(columns={0:'tfidf', 'level_0': 'document', 'level_1': 'term'})
    df2 = df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)
    tf_term = df2['term'].tolist()
    return tf_term

In [18]:
def find_tfidf2term(content):
    vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(2, 2))
    x = vectorizer.fit_transform(content)
    vectorizer.get_feature_names_out()
    df = pd.DataFrame(x.toarray(),  columns=vectorizer.get_feature_names_out())
    df = df.stack().reset_index()
    df = df.rename(columns={0:'tfidf', 'level_0': 'document', 'level_1': 'term'})
    df2 = df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(20)
    tf_term = df2['term'].tolist()
    return tf_term

In [19]:
def showtext(idx=0):
    text = [news_list[idx]['content']]
    t = find_tfidf(text)
    print("TF-IDF")
    print(t)
    print("\nSummary")
    print(news_list[idx]['summary'])

In [20]:
def showtext2term(idx=0):
    text = [news_list[idx]['content']]
    t = find_tfidf2term(text)
    print("TF-IDF")
    print(t)
    print("\nSummary")
    print(news_list[idx]['summary'])

In [21]:
def showtext_ent(idx):
    summary = news_list[idx]['summary']
    doc = nlp(summary)
    displacy.serve(doc, style="ent")
    print("=======================================")
    content = news_list[idx]['content']
    doc = nlp(content)
    displacy.serve(doc, style="ent")


In [22]:
def filter_and_sort_score(score_dict, num_sent=11):
    res = nlargest(num_sent, score_dict, key = score_dict.get)
    res.sort()
    #print(res)
    # return sorted list
    return res

In [23]:
def make_sentence(sent_list, sorted_score_dict):
    """
    sent_list : list of tokenized sentence of each article
    sorted_score_dict that we want to make sentence
    """
    #len_sent_list = len(sent_list)
    #print(len_sent_list)
    final_sent_list = []
    for i in sorted_score_dict:
        #print(i)
        if i <= len(sent_list)-1:
            final_sent_list.append(sent_list[i])

    # join into string
    #result = ' '.join(final_sent_list).replace(',', ' ')
    result = ' '.join(final_sent_list)
    return result

In [24]:
def make_sentence_coref(sent_list, sorted_score_dict, final_sent_list):
    """
    sent_list : list of tokenized sentence of each article
    sorted_score_dict that we want to make sentence
    """
    #len_sent_list = len(sent_list)
    #print(len_sent_list)
    for i in sorted_score_dict:
        #print(i)
        #if i <= len_sent_list-1:
        final_sent_list.append(sent_list[i])

    # join into string
    #result = ' '.join(final_sent_list).replace(',', ' ')
    result = ' '.join(final_sent_list)
    return result

In [25]:
text = news_list[0]['content']
text

"Marseille , France -LRB- CNN -RRB- The French prosecutor leading an investigation into the crash of Germanwings Flight 9525 insisted Wednesday that he was not aware of any video footage from on board the plane . Marseille prosecutor Brice Robin told CNN that `` so far no videos were used in the crash investigation . '' He added , `` A person who has such a video needs to immediately give it to the investigators . '' Robin 's comments follow claims by two magazines , German daily Bild and French Paris Match , of a cell phone video showing the harrowing final seconds from on board Germanwings Flight 9525 as it crashed into the French Alps . All 150 on board were killed . Paris Match and Bild reported that the video was recovered from a phone at the wreckage site . The two publications described the supposed video , but did not post it on their websites . The publications said that they watched the video , which was found by a source close to the investigation . `` One can hear cries of 

In [26]:
text = news_list[5]['content']
text

"-LRB- CNN -RRB- A Duke student has admitted to hanging a noose made of rope from a tree near a student union , university officials said Thursday . The prestigious private school did n't identify the student , citing federal privacy laws . In a news release , it said the student was no longer on campus and will face student conduct review . The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the noose on the tree early Wednesday , the university said . Officials are still trying to determine if other people were involved . Criminal investigations into the incident are ongoing as well . Students and faculty members marched Wednesday afternoon chanting `` We are not afraid . We stand together , '' after pictures of the noose were passed around on social media . At a forum held on the steps of Duke Chapel , close to where the noose was discovered at 2 a.m. , hundreds of people gathered . `` You came here for the re

In [27]:
tf = find_tfidf_idx(0)

In [28]:
ner_lst = nlp.pipe_labels['ner']
print(len(ner_lst))
print(ner_lst)

18
['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']


In [29]:
NER_list = ['DATE', 'EVENT', 'FAC', 'GPE', 'LOC', 'MONEY', 'NORP', 'ORG', 'PERSON',  'TIME',]

In [30]:
doc = nlp(text)
for ent in doc.ents:
    if ent.text.lower() in tf and ent.label_ in NER_list:
        print(ent.text,  ent.label_)

CNN ORG
CNN ORG


In [31]:
find_tfidf_idx(0)

['said',
 'lubitz',
 'cnn',
 'crash',
 'investigation',
 'told',
 'germanwings',
 'video',
 'flight',
 'site']

In [32]:
showtext_ent(11)

/Users/kneskung/opt/miniconda3/envs/pytorch/lib/python3.10/site-packages/spacy/displacy/__init__.py:94: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)


OSError: [Errno 48] Address already in use

In [33]:
news_list[11]['content']

"-LRB- CNN -RRB- Michele Bachmann is comparing President Obama to the co-pilot of the doomed Germanwings flight . `` With his Iran deal , Barack Obama is for the 300 million souls of the United States what Andreas Lubitz was for the 150 souls on the German Wings flight - a deranged pilot flying his entire nation into the rocks , '' the Minnesota Republican and former representative wrote in a Facebook comment posted March 31 . `` After the fact , among the smoldering remains of American cities , the shocked survivors will ask , why did he do it ? '' Andreas Lubitz , the co-pilot of Germanwings Flight 9525 , is accused by authorities of deliberately crashing the plane in the French Alps . He died in the crash along with 149 other crew and passengers . The motive of the March 24 crash is under investigation , though investigators are looking in to whether Lubitz feared a medical condition would cause him to lose his pilot 's license . Many comments posted on her Facebook page blasted the

### Experiment

In [34]:
idx = 37

In [35]:
text = news_list[idx]['content']

In [36]:
sent_list = sent_tokenize(text)
len_sent_list = len(sent_list)
first3_sent = sent_list[:3]
last_sent = sent_list[-1:]

In [37]:
select_sent = sent_list[3:len_sent_list-1]

In [38]:
select_text = ' '.join(select_sent)

In [39]:
select_text_coref = coref.resolve_coreferences(select_text)

/Users/kneskung/opt/miniconda3/envs/pytorch/lib/python3.10/site-packages/allennlp/modules/token_embedders/pretrained_transformer_embedder.py:385: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  num_effective_segments = (seq_lengths + self._max_length - 1) // self._max_length


In [40]:
select_text_coref

"`` Operating accelerators for the benefit of the physics community is what CERN 's here for , '' CERN Director-General Rolf Heuer said on the organization 's website . `` Today , CERN 's heart beats once more to the rhythm of the LHC . '' the LHC generates up to 600 million particles per second , with a beam circulating for 10 hours , traveling more than 6 billion miles -LRB- more than 10 billion kilometers -RRB- -- the distance from Earth to Neptune and back again . At near light-speed , a proton in the LHC makes 11,245 circuits per second . It took thousands of scientists , engineers and technicians decades to devise and build the particle accelerator , housed in a tunnel between Lake Geneva and the Jura mountain range . The purpose of the lengthy project is to recreate the conditions that existed moments after the `` Big Bang '' -- the scientific theory said to explain the creation of the universe . By replicating the energy density and temperature , scientists hope to uncover how 